In [4]:
"""
This script runs an evaluation of a LangGraph agent on a dataset using LangSmith's async evaluation API.
It converts dataset examples into the agent's internal state format, invokes the agent graph with unique checkpointing,
and uses a custom evaluator that leverages an LLM judge to compare the agent's output against reference answers.

Key components:
- example_to_state: converts dataset inputs to the agent's state format.
- target_with_config: async wrapper to invoke the agent graph with checkpoint config.
- correct: async evaluator function that prompts an LLM to judge correctness of the agent's answer.
- aevaluate: runs the evaluation over the dataset with concurrency and experiment tracking.
"""

#==============================================================================
# IMPORTS
#==============================================================================
import sys
import os

# Add the parent directory to the Python path so we can import the my_agent module
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

import uuid
from langsmith import aevaluate
from my_agent import create_graph
from my_agent.utils.state import DataAnalysisState
from my_agent.utils.models import get_azure_llm

#==============================================================================
# INITIALIZATION
#==============================================================================
# Get Model
judge_llm = get_azure_llm(temperature=0.0)

# Create the agent graph
graph = create_graph()

#==============================================================================
# EVALUATION FUNCTIONS
#==============================================================================
async def correctness(outputs: dict, reference_outputs: dict) -> bool:
    """
    Evaluator function that uses an LLM to determine if the actual answer contains all information
    from the expected answer.

    Args:
        outputs (dict): The output dictionary from the agent, expected to contain a 'messages' list.
        reference_outputs (dict): The reference outputs from the dataset, expected to have 'answers' key.

    Returns:
        bool: True if the LLM judge determines the answer is correct, False otherwise.
    """
    if not outputs or "messages" not in outputs or not outputs["messages"]:
        return False
        
    actual_answer = outputs["messages"][-1].content
    expected_answer = reference_outputs.get("answers", "[NO EXPECTED ANSWER PROVIDED]")
    
    instructions = (
        "Given an actual answer and an expected answer, determine whether"
        " the actual answer contains all of the information in the"
        " expected answer. Respond with 'CORRECT' if the actual answer"
        " does contain all of the expected information and 'INCORRECT'"
        " otherwise. Do not include anything else in your response."
    )
    user_msg = f"ACTUAL ANSWER: {actual_answer}\n\nEXPECTED ANSWER: {expected_answer}"
    
    response = await judge_llm.ainvoke(
        [
            {"role": "system", "content": instructions},
            {"role": "user", "content": user_msg}
        ]
    )
    return response.content.upper() == "CORRECT"

#==============================================================================
# STATE CONVERSION FUNCTIONS
#==============================================================================
def example_to_state(inputs: dict) -> dict:
    """
    Converts dataset example inputs into the agent's internal DataAnalysisState format.

    Args:
        inputs (dict): Input dictionary from the dataset, expected to have 'question' key.

    Returns:
        DataAnalysisState: Initialized state object for the agent.
    """
    return DataAnalysisState(
        prompt=inputs['question'],
        messages=[],
        result="",
        iteration=0
    )

async def target_with_config(inputs: dict):
    """
    Async wrapper to invoke the agent graph with a unique checkpoint thread_id in the config.

    Args:
        inputs (dict): Input dictionary from the dataset.

    Returns:
        dict: The output from the agent graph invocation, including 'messages'.
    """
    state = example_to_state(inputs)
    config = {"configurable": {"thread_id": str(uuid.uuid4())}}
    return await graph.ainvoke(state, config=config)

#==============================================================================
# MAIN EVALUATION
#==============================================================================
experiment_results = await aevaluate(
    target_with_config,
    data="czsu agent",
    evaluators=[correctness],
    max_concurrency=4,
    experiment_prefix="test1",
)

print(f"Evaluation results: {experiment_results}")


View the evaluation results for experiment: 'test1-b6ab35e2' at:
https://smith.langchain.com/o/817120e4-bc86-484a-9c30-795e42f5bb9e/datasets/e585a306-7113-491d-ba41-bc03c09eb612/compare?selectedSessions=53c8ef77-1e4a-4b45-ba9e-ad8199651902




0it [00:00, ?it/s]

AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
Evaluation results: <AsyncExperimentResults test1-b6ab35e2>
Evaluation results: <AsyncExperimentResults test1-b6ab35e2>
